In [1]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)

import os
import numpy as np
import pandas as pd
import time
import torch
import pickle
import shap
import tensorboard

from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, classification_report
from functools import partial
import shutil
from tqdm.auto import tqdm

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import linkage, dendrogram

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_distances

from codes.models.data_form.DataForm import DataTransfo_1SNP
from codes.models.metrics import calculate_roc_auc

import featurewiz as gwiz

import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve

from codes.models.Decision_tree.utils import get_indice, get_name


Gradient of fc1.weight: tensor([[-0.0193,  0.0146,  0.0230,  0.0026, -0.0158,  0.0272, -0.0056,  0.0026,
         -0.0167, -0.0043],
        [ 0.0000, -0.0000, -0.0000, -0.0000,  0.0000, -0.0000,  0.0000, -0.0000,
          0.0000,  0.0000],
        [-0.1777,  0.1340,  0.2113,  0.0243, -0.1454,  0.2507, -0.0512,  0.0236,
         -0.1534, -0.0392],
        [-0.0791,  0.0596,  0.0940,  0.0108, -0.0647,  0.1115, -0.0228,  0.0105,
         -0.0682, -0.0174],
        [-0.2129,  0.1605,  0.2531,  0.0291, -0.1742,  0.3003, -0.0613,  0.0283,
         -0.1838, -0.0469]])
Gradient of fc1.bias: tensor([-0.0187,  0.0000, -0.1719, -0.0765, -0.2060])
Gradient of fc2.weight: tensor([[ 0.0980,  0.0000,  0.1750,  0.1817,  0.1875],
        [-0.0980, -0.0000, -0.1750, -0.1817, -0.1875]])
Gradient of fc2.bias: tensor([ 0.4002, -0.4002])


In [3]:
### framework constants:
model_type = 'decision_tree'
model_version = 'gradient_boosting'
test_name = '1_test_train_transfo_V1'
tryout = True # True if we are ding a tryout, False otherwise 
### data constants:
### data constants:
CHR = 6
SNP = 'rs12203592'
pheno_method = 'Abby' # Paul, Abby
rollup_depth = 4
ld = 'no'
Classes_nb = 2 #nb of classes related to an SNP (here 0 or 1)
vocab_size = None # to be defined with data
padding_token = 0
prop_train_test = 0.8
load_data = True
save_data = False
remove_none = True
decorelate = False
equalize_label = False
threshold_corr = 0.9
threshold_rare = 50
remove_rare = 'all' # None, 'all', 'one_class'
compute_features = True
padding = False
list_env_features = []
list_pheno_ids = None #list(np.load(f'/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/phewas/list_associations_snps/{SNP}_paul.npy'))

### data format
batch_size = 20
data_share = 1

##### model constants


##### training constants
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
dataT = DataTransfo_1SNP(SNP=SNP,
                         CHR=CHR,
                         method=pheno_method,
                         padding=padding,  
                         pad_token=padding_token, 
                         load_data=load_data, 
                         save_data=save_data, 
                         compute_features=compute_features,
                         prop_train_test=prop_train_test,
                         remove_none=remove_none,
                         equalize_label=equalize_label,
                         rollup_depth=rollup_depth,
                         decorelate=decorelate,
                         threshold_corr=threshold_corr,
                         threshold_rare=threshold_rare,
                         remove_rare=remove_rare, 
                         list_env_features=list_env_features,
                         data_share=data_share,
                         list_pheno_ids=list_pheno_ids,
                         ld=ld)
#patient_list = dataT.get_patientlist()


In [5]:
data, labels_patients, indices_env, name_envs, eids = dataT.get_tree_data(with_env=False, load_possible=True, only_relevant=False)
frequencies_ini = np.sum(data, axis=0)

In [6]:
np.sum(labels_patients==1) / np.sum(labels_patients==0)

In [13]:
if pheno_method == 'Abby':
    embedding_file_diseases = f'/gpfs/commons/datasets/controlled/ukbb-gursoylab/mstoll/Data_Files/Embeddings/Abby/embedding_abby_no_1_diseases.pth'
    weight_diseases = torch.load(embedding_file_diseases)
else:
    embedding_file_diseases = '/gpfs/commons/groups/gursoy_lab/pmeddeb/phenotype_embedding/node2vec_embeddings/emb_4d_100_OMOP__ukbb_omop_rolled_up_depth_4.pickle'
#weight_diseases = torch.load(embedding_file_diseases).detach().numpy()[1:]
    with open(embedding_file_diseases, 'rb') as file:
        weight_diseases = pickle.load(file)


In [18]:
mat_distance = cosine_distances(weight_diseases)

### Gather in clusters

In [19]:
indices_keep = frequencies_ini > 2000

In [20]:
n_clusters = 100
# Vos données
X =  weight_diseases

# Définir une fonction de distance cosinus
def custom_cosine_distance(X, Y=None):
    return cosine_distances(X, Y)

# Adapter KMeans pour utiliser la distance cosinus
class CustomKMeans(KMeans):
    def fit(self, X, y=None):
        self.distance_func = custom_cosine_distance
        return super().fit(X, y)

# Utilisation de CustomKMeans avec la distance cosinus
kmeans = CustomKMeans(n_clusters=n_clusters)

# Ajustement du modèle aux données
kmeans.fit(X)

# Obtention des étiquettes de cluster pour chaque vecteur
clusters = kmeans.labels_

# Obtention des centres de cluster
centers = kmeans.cluster_centers_


In [25]:
cluster_id = 3
indices_cluster = np.where(clusters==cluster_id)[0]
get_name(dataT, indices_cluster)


In [ ]:
nb = [np.sum(clusters==i) for i in range(n_clusters)]

In [ ]:
plt.plot(nb, 'o')

### Pipeline

In [ ]:
clusters[indices_keep] = np.arange(indices_keep.sum()) + n_clusters

In [ ]:
print(f'nombre clusters = {len(np.unique(clusters))})

In [ ]:
equalized = True
interest = False
keep = False
scaled = True
fusion = True
random = False

In [ ]:

if fusion:
    df = pd.DataFrame(data.T)    
    df['cluster'] = clusters
    grouped = df.groupby('cluster').sum()
    data_fusion = grouped.T.to_numpy()
    data_fusion[data_fusion!=0] = 1
    data_use, labels_use = data_fusion, labels_patients
else:
    data_use, labels_use = data, labels_patients



if interest:
    data_use, labels_use = data_use[:nb_patients_interest, :-1], labels_use[:nb_patients_interest]
else:
    data_use, labels_use = data_use, labels_use

if equalized:
    pheno, labels = DataTransfo_1SNP.equalize_label(data=data_use, labels = labels_use)
else:
    pheno, labels = data_use, labels_use


In [ ]:
#sparcity_fusion = np.sum(data_fusion==0) / np.size(data_fusion)
#sparcity_ini = np.sum(data==0) / np.size(data)
#sparcity_ini, sparcity_fusion


In [ ]:
diseases_patients_train, diseases_patients_test, label_patients_train, label_patients_test = train_test_split(pheno, labels, test_size = 1-prop_train_test, random_state=42)

In [ ]:
class_weight = {0: np.sum(label_patients_train == 1) / np.sum(label_patients_train == 0), 1: 1.0}


In [ ]:
frequencies = np.sum(diseases_patients_train, axis=0)
indices_keep = frequencies > 100
print(f'nb phenos keep = {indices_keep.sum()}')

In [ ]:
diseases_patients_train_keep, diseases_patients_test_keep = diseases_patients_train[:, indices_keep], diseases_patients_test[:, indices_keep]

In [ ]:
if keep:
    diseases_patients_train_model = diseases_patients_train_keep
    diseases_patients_test_model = diseases_patients_test_keep
else:
    diseases_patients_train_model = diseases_patients_train
    diseases_patients_test_model = diseases_patients_test

 
diseases_patients_train_model_unscaled = diseases_patients_train_model
diseases_patients_test_model_unscaled = diseases_patients_test_model

if scaled:

    scaler = StandardScaler()
    diseases_patients_train_model= scaler.fit_transform(diseases_patients_train_model)
    diseases_patients_test_model = scaler.fit_transform(diseases_patients_test_model)

if random:
    np.random.shuffle(label_patients_train)

In [ ]:
model = HistGradientBoostingClassifier(class_weight=class_weight)


# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(diseases_patients_train_model, label_patients_train)

# Faire des prédictions sur l'ensemble de test
labels_pred_test = model.predict(diseases_patients_test_model)
labels_pred_train = model.predict(diseases_patients_train_model)
proba_test = model.predict_proba(diseases_patients_test_model)[:, 1]
proba_train = model.predict_proba(diseases_patients_train_model)[:, 1]

In [ ]:
nb_positive_train = np.sum(labels_pred_train==0)
nb_negative_train = np.sum(labels_pred_train==1)
nb_positive_test = np.sum(labels_pred_test==0)
nb_negative_test = np.sum(labels_pred_test==1)

TP_test = np.sum((label_patients_test==0 )& (labels_pred_test == 0)) / nb_positive_test
FP_test = np.sum((label_patients_test==1 )& (labels_pred_test == 0)) / nb_positive_test
TN_test = np.sum((label_patients_test==1 )& (labels_pred_test == 1)) / nb_negative_test
FN_test = np.sum((label_patients_test== 0)& (labels_pred_test == 1)) / nb_negative_test

TP_train = np.sum((label_patients_train==0 )& (labels_pred_train == 0)) / nb_positive_train
FP_train = np.sum((label_patients_train==1 )& (labels_pred_train == 0)) / nb_positive_train
TN_train = np.sum((label_patients_train==1 )& (labels_pred_train == 1)) / nb_negative_train
FN_train = np.sum((label_patients_train== 0)& (labels_pred_train == 1)) / nb_negative_train

accuracy_train = accuracy_score(label_patients_train, labels_pred_train)
accuracy_test = accuracy_score(label_patients_test, labels_pred_test)

auc_test = calculate_roc_auc(label_patients_test, proba_test)
auc_train = calculate_roc_auc(label_patients_train, proba_train)

proba_avg_zero_test = 1- np.mean(proba_test[label_patients_test==0])
proba_avg_zero_train = 1- np.mean(proba_train[label_patients_train==0])
proba_avg_one_test = np.mean(proba_test[label_patients_test==1])
proba_avg_one_train = np.mean(proba_train[label_patients_train==1])

In [ ]:
print(f'{TP_test=}') 
print(f'{FP_test=}')
print(f'{TN_test=}')
print(f'{FN_test=}')
print(f'{TP_train=}') 
print(f'{FP_train=}')
print(f'{TN_train=}')
print(f'{FN_train=}')
print(' ')
print(f'{auc_test=}')
print(f'{auc_train=}')
print(' ')
print(' ')
print(f'{accuracy_test=}')
print(f'{accuracy_train=}')
print(' ')
print(f'{proba_avg_zero_test=}')
print(f'{proba_avg_zero_train=}')
print(f'{proba_avg_one_test=}')
print(f'{proba_avg_one_train=}')

####  Plot according to frequencies

In [ ]:
diseases_patients_train.shape

In [ ]:
diseases_patients_train.shape

In [ ]:
frequencies = np.linspace(0, 4000, 20)
aucs = []
nb_phenos = []
for frequency in frequencies:
    print(frequency)
    indices_keep = frequencies_ini <= frequency
    nb_pheno = indices_keep.sum()
    diseases_patients_train_keep, diseases_patients_test_keep = diseases_patients_train[:, indices_keep], diseases_patients_test[:, indices_keep]
    model = HistGradientBoostingClassifier(class_weight=class_weight)


    # Entraîner le modèle sur l'ensemble d'entraînement
    model.fit(diseases_patients_train_keep, label_patients_train)

    # Faire des prédictions sur l'ensemble de test
    labels_pred_test = model.predict(diseases_patients_test_keep)
    labels_pred_train = model.predict(diseases_patients_train_keep)
    proba_test = model.predict_proba(diseases_patients_test_keep)[:, 1]
    proba_train = model.predict_proba(diseases_patients_train_keep)[:, 1]

    aucs.append(calculate_roc_auc(label_patients_test, proba_test))
    nb_phenos.append(nb_pheno)

    
 

In [ ]:
plt.plot(frequencies, aucs, 'o')
plt.xlabel('frequencies')
plt.ylabel('auc val')

### Calibration Plot

In [ ]:
predicted_probs_ones = proba_train
true_labels = np.array(label_patients_train)
plt.hist(predicted_probs_ones, bins=100)
plt.xlabel('proba')
plt.ylabel('nb of predictions')
plt.show()

prob_true, prob_pred = calibration_curve(true_labels, predicted_probs_ones, n_bins=80)
auc = calculate_roc_auc(true_labels, predicted_probs_ones)
# Tracer le graphique de calibration
plt.plot(prob_pred, prob_true, marker='o', linestyle='--', label='Calibration Plot')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfectly Calibrated')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.title('Calibration Plot')
plt.legend()

### Nb diseases

In [ ]:
nb_maladies_train = diseases_patients_train_model_unscaled.sum(axis=1)
nb_maladies_test = diseases_patients_test_model_unscaled.sum(axis=1)

In [ ]:
accs_train = []
nbs_train = []
for diseases_nb in np.unique(nb_maladies_train):
    patients_nb_diseases = diseases_patients_train_model[nb_maladies_train == diseases_nb]
    labels_nb_diseases = label_patients_train[nb_maladies_train == diseases_nb]
    labels_nb_diseases_pred = labels_pred_train[nb_maladies_train == diseases_nb]
    acc = np.sum(labels_nb_diseases == labels_nb_diseases_pred ) / len(labels_nb_diseases)
    accs_train.append(acc)
    nbs_train.append(len(labels_nb_diseases))
nb_diseases_mean_train = np.mean(nb_maladies_train)
print(f'nb diseases mean train= {nb_diseases_mean_train}')

accs_test = []
nbs_test = []
for diseases_nb in np.unique(nb_maladies_test):
    patients_nb_diseases = diseases_patients_test_model[nb_maladies_test == diseases_nb]
    labels_nb_diseases = label_patients_test[nb_maladies_test == diseases_nb]
    labels_nb_diseases_pred = labels_pred_test[nb_maladies_test == diseases_nb]
    acc = np.sum(labels_nb_diseases == labels_nb_diseases_pred ) / len(labels_nb_diseases)
    accs_test.append(acc)
    nbs_test.append(len(labels_nb_diseases))
nb_diseases_mean_test = np.mean(nb_maladies_test)
print(f'nb diseases mean test= {nb_diseases_mean_test}')


In [ ]:
color_values_nbs_train = nbs_train
color_values_nbs_test = nbs_test


In [ ]:
plt.scatter(np.unique(nb_maladies_train), accs_train,  c= color_values_nbs_train , cmap='viridis')
plt.xlabel('nb of diseases')
plt.ylabel('accuracy')
plt.colorbar()

In [ ]:
plt.scatter(np.unique(nb_maladies_test), accs_test,  c= color_values_nbs_test , cmap='viridis')
plt.xlabel('nb of diseases')
plt.ylabel('accuracy')
plt.colorbar()

### Shap values

In [ ]:
frequencies = np.sum(diseases_patients_train_model_unscaled, axis=0)
log_freq = np.log(frequencies+1)
color_values = log_freq



In [ ]:
explainer = shap.TreeExplainer(model)

# Compute SHAP values for a set of samples (e.g., X_test)
shap_values = explainer.shap_values(diseases_patients_train_model)

# Plot the SHAP values
#shap.summary_plot(shap_values, diseases_patients_test)

In [ ]:
shaps = np.abs(shap_values).mean(axis=0)
plt.scatter(np.arange(len(shaps)), shaps,  c= color_values , cmap='viridis')
plt.xlabel('phenotypes')
plt.ylabel('shap values')
plt.colorbar()


In [ ]:
np.all(diseases_patients_train_keep==0, axis=1).sum()

In [ ]:
indices_keep = shaps> 0.004
print(f'nb patients keep = {indices_keep.sum()}')

In [ ]:
indices = np.argsort(shaps)[::-1]

In [ ]:
indices[:15]

In [ ]:
accuracy_pheno_par(phenos)[64]

In [ ]:
cluster_id = 297
indices_cluster = np.where(clusters==cluster_id)[0]
get_name(dataT, indices_cluster)

### score phenos

In [ ]:
frequencies = np.sum(diseases_patients_train_model_unscaled, axis=0)
log_freq = np.log(frequencies+1)
color_values = log_freq



In [ ]:
def get_accuracy_pheno(data, labels_true, labels_pred, nb_pheno):
    coherence = labels_true[data[:,nb_pheno-1]==1] == labels_pred[data[:,nb_pheno-1]==1]
    accuracy_pheno = np.sum(coherence)/ len(coherence)
    return accuracy_pheno
accuracy_pheno_par = partial(get_accuracy_pheno, diseases_patients_test_model_unscaled,  label_patients_test, labels_pred_test)
phenos = np.arange(1, diseases_patients_train_model.shape[1]+1)
accuracy_pheno_par = np.vectorize(accuracy_pheno_par)

In [ ]:
plt.scatter(np.arange(len(phenos)), accuracy_pheno_par(phenos),  c= color_values , cmap='viridis')
plt.colorbar()

### FT - Growth

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
import pandas as pd

# Example transactions with integers
transactions = [
    [1, 2, 3, 4],
    [1, 2],
    [1, 2, 3, 4],
    [1],
    [1, 2, 5]
]

# Convert transactions to transaction encoding format
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

# Convert to DataFrame
df = pd.DataFrame(te_ary, columns=range(1, te_ary.shape[1]+1))

# Find frequent itemsets using FP-Growth
frequent_itemsets = fpgrowth(df, min_support=0.4, use_colnames=True)

print("Frequent itemsets:")
print(frequent_itemsets)


In [ ]:
data_ass = data.astype(bool)

In [ ]:
indices_keep.sum()

In [ ]:
data_ass_one = data_ass[labels_patients==1]
data_ass_zero= data_ass[labels_patients==0]

In [ ]:
data_ass_one_r = data_ass_one[:, indices_keep]
data_ass_zero_r = data_ass_zero[:, indices_keep]

In [ ]:
# Convert to DataFrame
df_1 = pd.DataFrame(data_ass_one_r, columns=range(1, data_ass_one_r.shape[1]+1))
df_0 = pd.DataFrame(data_ass_zero_r, columns=range(1, data_ass_zero_r.shape[1]+1))

In [ ]:
from mlxtend.frequent_patterns import fpgrowth


In [ ]:
frequent_itemsets_1 = fpgrowth(df_1, min_support=0.001, use_colnames=True)


In [ ]:
frequent_itemsets_0 = fpgrowth(df_0, min_support=0.001, use_colnames=True)


In [ ]:
data_ass_one_r.shape, data_ass_zero_r.shape

In [ ]:
frequent_itemsets_0[frequent_itemsets_0['support']> 0.05].sort_values('support', ascending=False)

In [ ]:
frequent_itemsets_1[frequent_itemsets_1['support']> 0.05].sort_values('support', ascending=False)

In [ ]:
merged_df = frequent_itemsets_1.merge(frequent_itemsets_0, left_on='itemsets', right_on='itemsets').sort_values('support_x')


In [ ]:
support_zero, support_one = np.array(merged_df['support_y']), np.array(merged_df['support_x'])

In [ ]:
merged_df

In [ ]:
support_zero, support_one   

In [ ]:
res = support_one / support_zero
res = np.max(np.concatenate([res, res**-1], axis=0).reshape(2, res.shape[0]), axis=0)
plt.plot(res, 'o')

In [ ]:
merged_df.loc[3972]

In [ ]:
indices = res > 4

In [ ]:
merged_df.loc[indices]

In [ ]:
indices = data_ass[:, [16, 296]].sum(axis=1) ==2

In [ ]:
(labels_patients[indices]==1).sum()

In [ ]:
(labels_patients[indices]==0).sum()

In [ ]:
arg = np.argmax(res)

In [ ]:
merged_df['itemsets'][arg]

In [ ]:
support_one[arg], support_zero[arg]

In [ ]:
frequent_itemsets_0

In [ ]:
frequent_itemsets_1

In [ ]:
items = frequent_itemsets_1.itemsets
explode = items.explode()

In [ ]:
dic_map = {}
for item in items:
    dic_map[item] = explode